In [6]:
'''

CINEMAC. 

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.

project for the course of Network Analysis - 2022/2023
Realized by.
	Andrea D'Arpa
	Maddalena Ghiotto		
	Chloe Papadopoulou

'''
import os
import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
from ast import literal_eval
from listReader import *
%pip install unidecode
from unidecode import unidecode






Note: you may need to restart the kernel to use updated packages.


**Firstly, import and read the necessary datasets for the project.**


**title.principals.tsv: Contains the principal cast/crew for titles**
tconst (string) - alphanumeric unique identifier of the title
ordering (integer) – a number to uniquely identify rows for a given titleId
nconst (string) - alphanumeric unique identifier of the name/person
category (string) - the category of job that person was in
job (string) - the specific job title if applicable, else '\N'

**name.basics.tsv – Contains the following information for names of people:**
nconst (string) - alphanumeric unique identifier of the name/person
primaryName (string)– name by which the person is most often credited
birthYear – in YYYY format
deathYear – in YYYY format if applicable, else '\N'
primaryProfession (array of strings)– the top-3 professions of the person
knownForTitles (array of tconsts) – titles the person is known for.


**title.basics.tsv contains all available title information from imdb:**
tconst (string) - alphanumeric unique identifier of the title
titleType (string) – the type/format of the title (e.g. movie, short, tvseries, tvepisode, video, etc)
primaryTitle (string) – the more popular title / the title used by the filmmakers on promotional materials at the point of release
originalTitle (string) - original title, in the original language
isAdult (boolean) - 0: non-adult title; 1: adult title
startYear (YYYY) – represents the release year of a title. In the case of TV Series, it is the series start year
endYear (YYYY) – TV Series end year. ‘\N’ for all other title types
runtimeMinutes – primary runtime of the title, in minutes
genres (string array) – includes up to three genres associated with the title.

As the focus of the project is on movies, other formats will be excluded (besides the values 'movie', 'tvMovie', and 'short'.)

**movie-links.gz - file containing their movies and their citations from a repository...**
**countries.gz - file containing movies and their country of origin;** in some cases, there is more than one value.

Using the functions imported from listreader, the last two files will be furtherly parsed in order to create the appropriate dataframes, as in their original state they only contain values in text format. Remember to copyright the people of the other project -- link--



In [7]:

# creating the tables with pandas to interact with our datasets
# getting the current working directory
cwd = os.getcwd()
path ="/".join(list(cwd.split('/')[0:-1])) 

name = path+'/datasets/name.basics.tsv'

name_table = pd.read_csv(name , sep='\t', header=0)

title = path+'/datasets/title.principals.tsv'
title_table = pd.read_csv(title , sep='\t', header=0)

#import titles ds to match the movie-links titles, 
title_name = path+'/datasets/title.basics.tsv'
title_name_table= pd.read_csv(title_name, sep='\t', header=0, low_memory=False)


countries = path+'/datasets/countries.list.gz'   #get countries
countries_table = get_movie_countries(countries)

movies = path+'/datasets/movie-links.list.gz'
movies_table = get_movie_links(movies)

'''
testing the tables
'''
# print(movies_table.head(5))
# print(countries_table.head(5))
# print(title_table.head(5))
# print(name_table.head(5))


#merge films and citations with their corresponding countries:

movies_cit_countries = pd.merge(movies_table, countries_table, on='movie', how='left')

#perform second merge for the movie's citations
movies_cit_countries=pd.merge(movies_cit_countries, countries_table, left_on="cites", right_on="movie",how="left")

movies_cit_countries.rename(columns={"movie_x":"movie","movie_y":"cited_movies","country_x":"movie_country","country_y":"cites_country"},inplace=True)
movies_cit_countries=movies_cit_countries.reindex(columns= ['movie', 'movie_country', 'cites', 'movie_y','cites_country'])
movies_cit_countries.drop(columns=['movie_y'], inplace=True)
movies_cit_countries.head(100)

,movie,movie_country,cites,cites_country
0,#DevinAuditions (2016),Canada,Alien (1979),UK
1,#DevinAuditions (2016),Canada,Alien (1979),USA
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA
...,...,...,...,...
95,'Joe Strummer': The Man (2007),USA,Joe Strummer: The Future Is Unwritten (2007),UK
96,'Kurenai no kenjû' yo eien ni (2000),Japan,Kurenai no kenju (1961),Japan
97,'Men Olsenbanden var ikke død!' (1984),Norway,Olsen-banden møter kongen og knekten (1974),Norway
98,'Men Olsenbanden var ikke død!' (1984),Norway,Olsen-banden og Dynamitt-Harry (1970),Norway


In [8]:
movies_cit_countries['movie_country'].unique()
movies_cit_countries['cites_country'].unique()

array(['UK', 'USA', 'Germany', 'Hong Kong', 'New Zealand', 'Australia',
       'Netherlands', 'India', 'West Germany', 'Finland', 'Poland',
       'France', 'Italy', 'Philippines', 'Mexico', 'Ireland', 'Japan',
       'Norway', 'Greece', 'Jamaica', 'Belgium', 'Spain', 'Soviet Union',
       'Canada', 'South Korea', 'Argentina', 'Bolivia', 'Sweden',
       'Bosnia and Herzegovina', 'Austria', 'Turkey', 'Vietnam',
       'East Germany', 'Taiwan', 'Czechoslovakia', 'Yugoslavia',
       'Czech Republic', 'Bahamas', 'Iran', 'Ukraine', nan, 'Singapore',
       'Thailand', 'Brazil', 'Switzerland', 'China', 'Denmark', 'Romania',
       'South Africa', 'Israel', 'Cambodia', 'Indonesia', 'Panama',
       'Portugal', 'Pakistan', 'Chile', 'Peru', 'Estonia', 'Hungary',
       'Russia', 'Ghana', 'Burkina Faso', 'Malaysia', 'Luxembourg',
       'United Arab Emirates', 'Morocco', 'Monaco', 'Mauritius', 'Serbia',
       'Albania', 'Senegal', 'Latvia', 'Bulgaria',
       'Federal Republic of Yugoslavia'

To merge data coming from different sources, the cleaning process includes:
<li> replacing "\\N" with np.nan values in the imdb datasets.
<li> extracting values from the 'movie' column of the movie-links dataset  and storing title and year in separate columns
<li> further cleaning on the titles: removing leading and trailing spaces, normalizing values by removing special characters.<br>
We also identified some differences in titles of non-english titled movies; some were fixed but unfortunately a large part of them was dropped.
<li> removing data from both datasets regarding non-movie or short movie titles.

In [9]:
#retrieve information for movies collaborators

# replace /n for nan values in the imdb data
title_name_table = title_name_table.replace("\\N", np.nan)

# extract titles and year from the 'movie' column: pattern of four digits between parentheses at the end of the string
pattern = r'^(.*) \((\d{4})\)$'

movies_cit_countries[['title', 'year']] = movies_cit_countries['movie'].str.extract(pattern)
movies_cit_countries[['primaryTitle_citation', 'year_citation']] = movies_cit_countries['cites'].str.extract(pattern)

movies_cit_countries.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 332925 entries, 0 to 332924
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype 
---  ------                 --------------   ----- 
 0   movie                  332925 non-null  object
 1   movie_country          332634 non-null  object
 2   cites                  332925 non-null  object
 3   cites_country          332867 non-null  object
 4   title                  332925 non-null  object
 5   year                   332925 non-null  object
 6   primaryTitle_citation  332925 non-null  object
 7   year_citation          332925 non-null  object
dtypes: object(8)
memory usage: 22.9+ MB


In [10]:
#******TROUBLESHOOTING missing values on merge ********

#check encoding is same in both, utf-8 
#check that year format is correct everywhere
non_valid_year = list([y for y in movies_cit_countries['year'].unique() if not (str(y).isdigit() and len(str(y)) == 4)])
non_valid_year
error_rows = movies_cit_countries[movies_cit_countries['year'].isin(non_valid_year)]
error_rows

non_valid_year_x = list([y for y in movies_cit_countries['year_citation'].unique() if not (str(y).isdigit() and len(str(y)) == 4)])
non_valid_year_x
error_rows_x = movies_cit_countries[movies_cit_countries['year_citation'].isin(non_valid_year_x)]
error_rows.empty and error_rows_x.empty



True

In [11]:
#remove useless whitespaces
movies_cit_countries['title'] = movies_cit_countries['title'].str.strip()
movies_cit_countries['primaryTitle_citation'] = movies_cit_countries['primaryTitle_citation'].str.strip()

title_name_table['primaryTitle'] = title_name_table['primaryTitle'].str.strip()
title_name_table['originalTitle'] = title_name_table['originalTitle'].str.strip()

#take out tv series and videogames/videos

title_name_table = title_name_table[title_name_table["titleType"] == 'movie' ]
# title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvSeries' ]
# title_name_table = title_name_table.loc[title_name_table["titleType"] != 'videoGame' ]
# title_name_table = title_name_table.loc[title_name_table["titleType"] != 'video' ]
# title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvMiniSeries' ]
# title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvShort' ]
# title_name_table = title_name_table.loc[title_name_table["titleType"] != 'tvSpecial' ]



title_name_table.drop(columns=['endYear','runtimeMinutes'], inplace=True)
title_name_table

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0,1905,NaN
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,Drama
...,...,...,...,...,...,...,...
9640892,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015,Documentary
9640919,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007,Documentary
9640931,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013,Comedy
9640941,tt9916730,movie,6 Gunn,6 Gunn,0,2017,NaN


In [12]:
#replace a repeated occurence where we noticed typos
movies_cit_countries = movies_cit_countries.replace(['Zui jia pai dang 3: Nv huang mi ling'] ,'Zui jia pai dang 3: Nu huang mi ling')

#replace West germany 
movies_cit_countries['movie_country'] = movies_cit_countries['movie_country'].replace('West Germany', 'Germany')
movies_cit_countries['movie_country'] = movies_cit_countries['movie_country'].replace('East Germany', 'Germany')

movies_cit_countries['cites_country'] = movies_cit_countries['cites_country'].replace('West Germany', 'Germany')
movies_cit_countries['cites_country'] = movies_cit_countries['cites_country'].replace('East Germany', 'Germany')


In [13]:

#remove special characters etc

def normalize_string(s):
    s = s.replace('&', 'and')
    s = re.sub(r'[^\w\s]', '', s).lower()  # Keep alphanumeric and whitespace characters
    s = re.sub(r'\s+', '', s)  # Remove whitespace characters
    s = unidecode(s)  # Remove accents and diacritics
    return s


title_name_table['originalTitle_normalized'] = title_name_table['originalTitle'].apply(normalize_string)
movies_cit_countries['title_citation_normalized'] = movies_cit_countries['primaryTitle_citation'].apply(normalize_string)
title_name_table['primaryTitle_normalized'] = title_name_table['primaryTitle'].apply(normalize_string)


movies_cit_countries 

#make sure formats are the same
#assign best datatypes

movies_cit_countries=movies_cit_countries.convert_dtypes()
title_name_table=title_name_table.convert_dtypes()

#they all turn out as str- edit years
#remove nan values

movies_cit_countries['year'] =  movies_cit_countries['year'].fillna('0')
movies_cit_countries['year_citation'] =  movies_cit_countries['year_citation'].fillna('0')
title_name_table['startYear'] =  title_name_table['startYear'].fillna('0')


#assign types
movies_cit_countries= movies_cit_countries.astype({'year': 'int32', 'year_citation':'int32'})

title_name_table= title_name_table.astype({'startYear': 'int32'})



In [14]:
movies_cit_countries.info()
title_name_table.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 332925 entries, 0 to 332924
Data columns (total 9 columns):
 #   Column                     Non-Null Count   Dtype 
---  ------                     --------------   ----- 
 0   movie                      332925 non-null  string
 1   movie_country              332634 non-null  string
 2   cites                      332925 non-null  string
 3   cites_country              332867 non-null  string
 4   title                      332925 non-null  string
 5   year                       332925 non-null  int32 
 6   primaryTitle_citation      332925 non-null  string
 7   year_citation              332925 non-null  int32 
 8   title_citation_normalized  332925 non-null  string
dtypes: int32(2), string(7)
memory usage: 22.9 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 636664 entries, 8 to 9640951
Data columns (total 9 columns):
 #   Column                    Non-Null Count   Dtype 
---  ------                    --------------   ----- 
 0 

---------------------------------------------------------------------------------------------------------------------------
The first outer merge concerns the citations of the films, and connects movie data based on movie title; then the values are furtherly filtered with the condition that their year matches by a maximum difference of 3 years. This approach helps to a. identify the unmatched values and b. avoid a lot of missing values given the fact that our data comes from different sources and at times contains year differences.


In [15]:
#first merge on title 
final_df = pd.merge(movies_cit_countries, title_name_table,
             left_on=['title_citation_normalized'],
             right_on=['originalTitle_normalized'],how='outer', indicator=True)
                              

final_df.info()                     

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1172394 entries, 0 to 1172393
Data columns (total 19 columns):
 #   Column                     Non-Null Count    Dtype   
---  ------                     --------------    -----   
 0   movie                      589674 non-null   string  
 1   movie_country              589118 non-null   string  
 2   cites                      589674 non-null   string  
 3   cites_country              589600 non-null   string  
 4   title                      589674 non-null   string  
 5   year                       589674 non-null   float64 
 6   primaryTitle_citation      589674 non-null   string  
 7   year_citation              589674 non-null   float64 
 8   title_citation_normalized  589674 non-null   string  
 9   tconst                     1078889 non-null  string  
 10  titleType                  1078889 non-null  string  
 11  primaryTitle               1078889 non-null  string  
 12  originalTitle              1078889 non-null  string  
 1

In [16]:
#identify which are the movies that share the same year and filter
matches = final_df[final_df['_merge'] == "both"].copy()

#include 3 year differences
matches['year_diff'] = matches['year_citation'] - matches['startYear']
matches = matches.loc[matches['year_diff'].between(-3, 3)]

#now we have the more accurate matches
              

In [17]:
#find missing values
movies_cit_countries['movie'].unique()


<StringArray>
[                                     '#DevinAuditions (2016)',
                                        '#FromJennifer (2017)',
                                                 '#Rip (2013)',
                                         '#TubeClash02 (2016)',
 '#chicagoGirl: The Social Network Takes on a Dictator (2013)',
                                        '#twitterkills (2014)',
                                                '#will (2016)',
                                            '$10 Raise (1935)',
                               '$10,000 Under a Pillow (1921)',
                                             '$5 a Day (2008)',
 ...
                                'Úristen, itt lönek... (2005)',
                                       'Üksindusfilm 2 (2016)',
                                           'Üvegtigris (2001)',
                                        'Üvegtigris 2. (2006)',
                                        'Üvegtigris 3. (2010)',
                     

In [18]:
matches['primaryTitle_citation'].unique() #there are 5.000 values missing

<StringArray>
[                           'Alien',                         'Die Hard',
         "Ferris Bueller's Day Off",                       'Fight Club',
                       'Magic Mike',                    'The Gold Rush',
                      'The Shining',                       '2 Jennifer',
                      'To Jennifer',                     'Love, Simple',
 ...
                      'Syncopation',                 'The New Recruits',
                'Með allt á hreinu',              'Plecarea Vlasinilor',
 'Ôgon no taka - Zempen: Makyô-hen',                   'Taii no musume',
                       'Üvegtigris',                    'Üvegtigris 2.',
               'Letto a tre piazze',                      'Il successo']
Length: 31660, dtype: string

Next step is to investigate which films were not merged, and perform a second merge based on their title from the movies dataframe, and their title in the imdb dataset, but **this time in its primaryTitle format,** aka the titme that is often more commonly used.

In [19]:

imdb_unmatched = final_df[final_df['_merge'] == "right_only"].copy()
links_unmatched = final_df[final_df['_merge'] == "left_only"].copy()

imdb_unmatched
matches_1 = pd.merge(links_unmatched, imdb_unmatched,
             left_on=['title_citation_normalized'],
             right_on=['primaryTitle_normalized'],how='outer', indicator="match_primaryTitle") #match with primaryTitle this time

matches_1


,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,tconst_x,...,titleType_y,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle
0,"$10,000 Under a Pillow (1921)",USA,Bud Takes the Cake (1920),USA,"$10,000 Under a Pillow",1921.0,Bud Takes the Cake,1920.0,budtakesthecake,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
1,Bud and Susie Join the Tecs (1920),USA,Bud Takes the Cake (1920),USA,Bud and Susie Join the Tecs,1920.0,Bud Takes the Cake,1920.0,budtakesthecake,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
2,Bud and Tommy Take a Day Off (1920),USA,Bud Takes the Cake (1920),USA,Bud and Tommy Take a Day Off,1920.0,Bud Takes the Cake,1920.0,budtakesthecake,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
3,By the Sea (1921),USA,Bud Takes the Cake (1920),USA,By the Sea,1921.0,Bud Takes the Cake,1920.0,budtakesthecake,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
4,Circumstantial Evidence (1921),USA,Bud Takes the Cake (1920),USA,Circumstantial Evidence,1921.0,Bud Takes the Cake,1920.0,budtakesthecake,<NA>,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676213,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015.0,Documentary,rodolphoteophiloolegadodeumpioneiro,rodolphoteophiloolegadodeumpioneiro,right_only,right_only
676214,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007.0,Documentary,delailusionaldesconciertocinecolombiano19701995,delailusionaldesconciertocinecolombiano19701995,right_only,right_only
676215,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,movie,Dankyavar Danka,Dankyavar Danka,0,2013.0,Comedy,dankyavardanka,dankyavardanka,right_only,right_only
676216,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,<NA>,...,movie,6 Gunn,6 Gunn,0,2017.0,<NA>,6gunn,6gunn,right_only,right_only


In [20]:
matches_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 676218 entries, 0 to 676217
Data columns (total 39 columns):
 #   Column                       Non-Null Count   Dtype   
---  ------                       --------------   -----   
 0   movie_x                      93632 non-null   string  
 1   movie_country_x              93595 non-null   string  
 2   cites_x                      93632 non-null   string  
 3   cites_country_x              93603 non-null   string  
 4   title_x                      93632 non-null   string  
 5   year_x                       93632 non-null   float64 
 6   primaryTitle_citation_x      93632 non-null   string  
 7   year_citation_x              93632 non-null   float64 
 8   title_citation_normalized_x  93632 non-null   string  
 9   tconst_x                     0 non-null       string  
 10  titleType_x                  0 non-null       string  
 11  primaryTitle_x               0 non-null       string  
 12  originalTitle_x              0 non-null     

In [21]:
matches_primaryTitle = matches_1[matches_1['match_primaryTitle'] == "both"].copy()
matches_primaryTitle.values[0]
matches_primaryTitle['year_diff_prim'] = matches_primaryTitle['year_citation_x'] - matches_primaryTitle['startYear_y']
matches_primaryTitle = matches_primaryTitle.loc[matches_primaryTitle['year_diff_prim'].between(-3, 3)]
matches_primaryTitle

,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,tconst_x,...,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle,year_diff_prim
170,(500) Days of Summer (2009),USA,Three Seasons (1999),Vietnam,(500) Days of Summer,2009.0,Three Seasons,1999.0,threeseasons,<NA>,...,Three Seasons,Ba mùa,0,1999.0,Drama,bamua,threeseasons,right_only,both,0.0
171,(500) Days of Summer (2009),USA,Three Seasons (1999),Vietnam,(500) Days of Summer,2009.0,Three Seasons,1999.0,threeseasons,<NA>,...,Three Seasons,Treis epohes,0,1996.0,"Drama,Romance",treisepohes,threeseasons,right_only,both,3.0
172,(500) Days of Summer (2009),USA,Three Seasons (1999),USA,(500) Days of Summer,2009.0,Three Seasons,1999.0,threeseasons,<NA>,...,Three Seasons,Ba mùa,0,1999.0,Drama,bamua,threeseasons,right_only,both,0.0
173,(500) Days of Summer (2009),USA,Three Seasons (1999),USA,(500) Days of Summer,2009.0,Three Seasons,1999.0,threeseasons,<NA>,...,Three Seasons,Treis epohes,0,1996.0,"Drama,Romance",treisepohes,threeseasons,right_only,both,3.0
174,Film Geek (2005),USA,Three Seasons (1999),Vietnam,Film Geek,2005.0,Three Seasons,1999.0,threeseasons,<NA>,...,Three Seasons,Ba mùa,0,1999.0,Drama,bamua,threeseasons,right_only,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85452,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0
85453,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0
85454,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0
85455,Yin yang lu shi si zhi shuang gui pai men (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi si zhi shuang gui pai men,2002.0,Troublesome Night 11,2001.0,troublesomenight11,<NA>,...,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",yamyeunglo11liugwailomin,troublesomenight11,right_only,both,0.0


In [22]:
links_missing_completely = matches_1[matches_1['match_primaryTitle'] == "left_only"].copy()#show non matched data
#to check:
links_missing_completely['primaryTitle_citation_x'].unique()

<StringArray>
[                    'Bud Takes the Cake',
            'Bud and Susie Join the Tecs',
           'Bud and Tommy Take a Day Off',
                   'Down the Mississippi',
                         'Getting Theirs',
                     "Handy Mandy's Goat",
           "Ma's Wipe Your Feet Campaign",
                         'Mice and Money',
                 'Romance and Rheumatism',
         "The Kids Find Candy's Catching",
 ...
                           'FJ contra FJ',
                                 'Dexter',
        'À Saint-Henri le cinq septembre',
                 'Écono mixte - Volume 1',
                 'Écono mixte - Volume 2',
                          'Une explosion',
 'Épisode de la bataille de Modder-River',
        "Épopée napoléonienne - L'Empire",
                           'Üksindusfilm',
                              'Smáfuglar']
Length: 8801, dtype: string

As we can see the missing values are often non english titles; There were lengthy efforts to match the remaining titles using similarity algorithms,
but due to them being very computationally demanding, given the limitations of this project we had to drop the remaining unrecognized titles.

In [23]:
#remove columns with all nan values after merging

matches = matches.dropna(axis=1, how='all')
matches_primaryTitle = matches_primaryTitle.dropna(axis=1, how='all')

#cleaning

matches_primaryTitle.columns = matches_primaryTitle.columns.str.replace("_y", "")
matches_primaryTitle.columns = matches_primaryTitle.columns.str.replace("_x","")
matches_primaryTitle = matches_primaryTitle.reset_index(drop=True)
matches = matches.reset_index(drop=True)
matches_primaryTitle.drop(columns=['_merge'], inplace=True)
matches.drop(columns=['_merge'], inplace=True)
matches_primaryTitle.rename(columns={'year_diff_prim' : 'year_diff'}, inplace= True)

matches


,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,year_diff
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,alien,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",alien,alien,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
237273,Üvegtigris 3. (2010),Hungary,Üvegtigris (2001),Hungary,Üvegtigris 3.,2010.0,Üvegtigris,2001.0,uvegtigris,tt0257290,movie,Glass Tiger,Üvegtigris,0,2001.0,Comedy,uvegtigris,glasstiger,0.0
237274,Üvegtigris 3. (2010),Hungary,Üvegtigris 2. (2006),Hungary,Üvegtigris 3.,2010.0,Üvegtigris 2.,2006.0,uvegtigris2,tt0476508,movie,Glass Tiger 2,Üvegtigris 2.,0,2006.0,Comedy,uvegtigris2,glasstiger2,0.0
237275,è solo questione di punti di vista (2012),Italy,Letto a tre piazze (1960),Italy,è solo questione di punti di vista,2012.0,Letto a tre piazze,1960.0,lettoatrepiazze,tt0054023,movie,Letto a tre piazze,Letto a tre piazze,0,1960.0,Comedy,lettoatrepiazze,lettoatrepiazze,0.0
237276,è solo questione di tempo (2013),Italy,Il successo (1963),Italy,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,tt0057540,movie,Il successo,Il successo,0,1963.0,Comedy,ilsuccesso,ilsuccesso,0.0


In [24]:
#create final df with matches
final_citations = pd.concat([matches,matches_primaryTitle],ignore_index=True)


final_citations['match_primaryTitle'].replace({'both': 'True'},inplace=True)
final_citations['match_primaryTitle'] = final_citations['match_primaryTitle'].cat.add_categories(['False']).fillna('False')
final_citations.drop(columns=['originalTitle_normalized','primaryTitle_normalized','title_citation_normalized'],inplace=True)
final_citations


,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,year_diff,match_primaryTitle
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238045,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
238046,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
238047,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
238048,Yin yang lu shi si zhi shuang gui pai men (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi si zhi shuang gui pai men,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True


In [25]:

final_citations.drop_duplicates(inplace=True)
final_citations.columns = final_citations.columns.str.replace("_citation", "_cit")

final_citations= final_citations.add_suffix('_cit')
final_citations.rename(columns={'movie_cit':'movie', 'movie_country_cit':'movie_country','title_cit':'title','year_cit':'year'} ,inplace=True)
final_citations.columns = final_citations.columns.str.replace("cites_", "")
final_citations.columns = final_citations.columns.str.replace("_cit_cit", "_cit")

final_citations


,movie,movie_country,cit,country_cit,title,year,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,primaryTitle_cit,originalTitle_cit,isAdult_cit,startYear_cit,genres_cit,year_diff_cit,match_primaryTitle_cit
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
2,.com for Murder (2002),USA,Alien (1979),UK,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
3,.com for Murder (2002),USA,Alien (1979),USA,.com for Murder,2002.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
4,30 Minutes or Less (2011),Germany,Alien (1979),UK,30 Minutes or Less,2011.0,Alien,1979.0,tt0078748,movie,Alien,Alien,0,1979.0,"Horror,Sci-Fi",0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238045,Yin yang lu shi jiu zhi Wo dui yan jian dao ye...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi jiu zhi Wo dui yan jian dao ye,2003.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
238046,Yin yang lu shi liu zhi hui dao wu xia shi dai...,Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi liu zhi hui dao wu xia shi dai,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
238047,Yin yang lu shi qi zhi jian fang you gui (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi qi zhi jian fang you gui,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True
238048,Yin yang lu shi si zhi shuang gui pai men (2002),Hong Kong,Troublesome Night 11 (2001),Hong Kong,Yin yang lu shi si zhi shuang gui pai men,2002.0,Troublesome Night 11,2001.0,tt0297441,movie,Troublesome Night 11,Yam yeung lo 11: Liu gwai lo min,0,2001.0,"Comedy,Horror",0.0,True


This concludes the final data regarding our citations. The final dataframe contains the title, year and country of movies and their citations, and the imdb info (originalTitle, primaryTitle, year, tconst identifier, type, and genre info from imdb. The last column match_primaryTitle signifies whether the values were merged based on the primaryTitle or not.
<br>
<br>
We now move on to merging the two datasets to get imdb info regarding the *movies* citing other movies, repeating the same process for a. normalizing the movie names and performing the two merges based on titles matching either in the original or the primary format, and their year of release.

In [26]:
#MERGE OF MOVIES AND CITATIONS
#normalize movie names

movies_cit_countries['movie_normalized'] = movies_cit_countries['title'].apply(normalize_string)


In [27]:
#merge 
movies_df = pd.merge(movies_cit_countries, title_name_table,
             left_on='movie_normalized',
             right_on='originalTitle_normalized', how='outer', indicator=True)

movies_df

,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,movie_normalized,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,_merge
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,Die Hard,1988.0,diehard,devinauditions,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,Ferris Bueller's Day Off,1986.0,ferrisbuellersdayoff,devinauditions,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,Fight Club,1999.0,fightclub,devinauditions,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1053471,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015.0,Documentary,rodolphoteophiloolegadodeumpioneiro,rodolphoteophiloolegadodeumpioneiro,right_only
1053472,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,De la ilusión al desconcierto: cine colombiano...,0,2007.0,Documentary,delailusionaldesconciertocinecolombiano19701995,delailusionaldesconciertocinecolombiano19701995,right_only
1053473,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916706,movie,Dankyavar Danka,Dankyavar Danka,0,2013.0,Comedy,dankyavardanka,dankyavardanka,right_only
1053474,<NA>,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,NaN,<NA>,NaN,tt9916730,movie,6 Gunn,6 Gunn,0,2017.0,<NA>,6gunn,6gunn,right_only


In [28]:
matches_movies = movies_df[movies_df['_merge'] == "both"].copy()
matches_movies['year_diff'] = matches_movies['year'] - matches_movies['startYear']
matches_movies = matches_movies.loc[matches_movies['year_diff'].between(-3, 3)]
matches_movies

,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,movie_normalized,...,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,_merge,year_diff
10,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,2 Jennifer,2016.0,2jennifer,fromjennifer,...,movie,From Jennifer,From Jennifer,0,2017.0,"Comedy,Horror,Thriller",fromjennifer,fromjennifer,both,0.0
11,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,To Jennifer,2013.0,tojennifer,fromjennifer,...,movie,From Jennifer,From Jennifer,0,2017.0,"Comedy,Horror,Thriller",fromjennifer,fromjennifer,both,0.0
13,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,"Love, Simple",2012.0,lovesimple,rip,...,movie,Rip,Rip,0,2010.0,Musical,rip,rip,both,3.0
14,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,"Love, Simple",2012.0,lovesimple,rip,...,movie,R.I.P,R.I.P,0,2011.0,Action,rip,rip,both,2.0
25,R.I.P. (2013),USA,Doctor Faustus (1967),UK,R.I.P.,2013.0,Doctor Faustus,1967.0,doctorfaustus,rip,...,movie,Rip,Rip,0,2010.0,Musical,rip,rip,both,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
470190,è solo questione di punti di vista (2012),Italy,"Un genio, due compari, un pollo (1975)",Italy,è solo questione di punti di vista,2012.0,"Un genio, due compari, un pollo",1975.0,ungenioduecompariunpollo,esoloquestionedipuntidivista,...,movie,è solo questione di punti di vista,è solo questione di punti di vista,0,2012.0,"Action,Adventure,Comedy",esoloquestionedipuntidivista,esoloquestionedipuntidivista,both,0.0
470191,è solo questione di punti di vista (2012),Italy,"Un genio, due compari, un pollo (1975)",France,è solo questione di punti di vista,2012.0,"Un genio, due compari, un pollo",1975.0,ungenioduecompariunpollo,esoloquestionedipuntidivista,...,movie,è solo questione di punti di vista,è solo questione di punti di vista,0,2012.0,"Action,Adventure,Comedy",esoloquestionedipuntidivista,esoloquestionedipuntidivista,both,0.0
470192,è solo questione di punti di vista (2012),Italy,"Un genio, due compari, un pollo (1975)",Germany,è solo questione di punti di vista,2012.0,"Un genio, due compari, un pollo",1975.0,ungenioduecompariunpollo,esoloquestionedipuntidivista,...,movie,è solo questione di punti di vista,è solo questione di punti di vista,0,2012.0,"Action,Adventure,Comedy",esoloquestionedipuntidivista,esoloquestionedipuntidivista,both,0.0
470193,è solo questione di tempo (2013),Italy,Il successo (1963),Italy,è solo questione di tempo,2013.0,Il successo,1963.0,ilsuccesso,esoloquestioneditempo,...,movie,è solo questione di tempo,è solo questione di tempo,0,2013.0,Comedy,esoloquestioneditempo,esoloquestioneditempo,both,0.0


In [29]:
imdb_unmatched_1 = movies_df[movies_df['_merge'] == "right_only"].copy()
links_unmatched_1 = movies_df[movies_df['_merge'] == "left_only"].copy()
imdb_unmatched_1

movies_candidate_primary = pd.merge(links_unmatched_1, imdb_unmatched_1,
             left_on=['movie_normalized'],
             right_on=['primaryTitle_normalized'],how='outer', indicator="match_primaryTitle")


movies_match_primary = movies_candidate_primary[movies_candidate_primary['match_primaryTitle'] == "both"].copy()
movies_match_primary['year_diff_prim'] = movies_match_primary['year_x'] - movies_match_primary['startYear_y']
movies_match_primary = movies_match_primary.loc[movies_match_primary['year_diff_prim'].between(-3, 3)]
movies_match_primary

,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,movie_normalized_x,...,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle,year_diff_prim
2789,A Real Life!!! (2006),Germany,La più bella serata della mia vita (1972),Italy,A Real Life!!!,2006.0,La più bella serata della mia vita,1972.0,lapiubellaseratadellamiavita,areallife,...,A Real Life,Au voleur,0,2009.0,"Crime,Drama,Romance",auvoleur,areallife,right_only,both,-3.0
2790,A Real Life!!! (2006),Germany,La più bella serata della mia vita (1972),France,A Real Life!!!,2006.0,La più bella serata della mia vita,1972.0,lapiubellaseratadellamiavita,areallife,...,A Real Life,Au voleur,0,2009.0,"Crime,Drama,Romance",auvoleur,areallife,right_only,both,-3.0
5628,Angels Wash Their Faces (1939),USA,Code of the Streets (1939),USA,Angels Wash Their Faces,1939.0,Code of the Streets,1939.0,codeofthestreets,angelswashtheirfaces,...,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,right_only,both,0.0
5629,Angels Wash Their Faces (1939),USA,Hell's Kitchen (1939),USA,Angels Wash Their Faces,1939.0,Hell's Kitchen,1939.0,hellskitchen,angelswashtheirfaces,...,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,right_only,both,0.0
5630,Angels Wash Their Faces (1939),USA,They Made Me a Criminal (1939),USA,Angels Wash Their Faces,1939.0,They Made Me a Criminal,1939.0,theymademeacriminal,angelswashtheirfaces,...,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,right_only,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102923,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0
102924,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0
102925,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0
102926,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,xfirstclass,xmendarkphoenix,...,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,right_only,both,-1.0


In [30]:
movies_candidate_primary[movies_candidate_primary['match_primaryTitle']=='left_only']


,movie_x,movie_country_x,cites_x,cites_country_x,title_x,year_x,primaryTitle_citation_x,year_citation_x,title_citation_normalized_x,movie_normalized_x,...,titleType_y,primaryTitle_y,originalTitle_y,isAdult_y,startYear_y,genres_y,originalTitle_normalized_y,primaryTitle_normalized_y,_merge_y,match_primaryTitle
0,#DevinAuditions (2016),Canada,Alien (1979),UK,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
1,#DevinAuditions (2016),Canada,Alien (1979),USA,#DevinAuditions,2016.0,Alien,1979.0,alien,devinauditions,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
2,#DevinAuditions (2016),Canada,Die Hard (1988),USA,#DevinAuditions,2016.0,Die Hard,1988.0,diehard,devinauditions,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
3,#DevinAuditions (2016),Canada,Ferris Bueller's Day Off (1986),USA,#DevinAuditions,2016.0,Ferris Bueller's Day Off,1986.0,ferrisbuellersdayoff,devinauditions,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
4,#DevinAuditions (2016),Canada,Fight Club (1999),USA,#DevinAuditions,2016.0,Fight Club,1999.0,fightclub,devinauditions,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103763,Ília (2014),Brazil,Nymphomaniac: Vol. I (2013),Belgium,Ília,2014.0,Nymphomaniac: Vol. I,2013.0,nymphomaniacvoli,ilia,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
103764,Ília (2014),Brazil,Nymphomaniac: Vol. I (2013),UK,Ília,2014.0,Nymphomaniac: Vol. I,2013.0,nymphomaniacvoli,ilia,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
103765,Ília (2014),Brazil,Nymphomaniac: Vol. I (2013),France,Ília,2014.0,Nymphomaniac: Vol. I,2013.0,nymphomaniacvoli,ilia,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only
103766,"Úristen, itt lönek... (2005)",Hungary,"Kelj fel, komám, ne aludjál (2002)",Hungary,"Úristen, itt lönek...",2005.0,"Kelj fel, komám, ne aludjál",2002.0,keljfelkomamnealudjal,uristenittlonek,...,<NA>,<NA>,<NA>,<NA>,NaN,<NA>,<NA>,<NA>,NaN,left_only


In [31]:
#check missing values
matches_movies = matches_movies.dropna(axis=1, how='all')
movies_match_primary = movies_match_primary.dropna(axis=1, how='all')

movies_match_primary.columns = movies_match_primary.columns.str.replace("_y", "")
movies_match_primary.columns = movies_match_primary.columns.str.replace("_x","")

matches_movies = matches_movies.reset_index(drop=True)
movies_match_primary = movies_match_primary.reset_index(drop=True)
# matches_movies.drop(columns=['_merge'], inplace=True)
movies_match_primary.drop(columns=['_merge'], inplace=True)
movies_match_primary.rename(columns={'year_diff_prim' : 'year_diff'}, inplace= True)

movies_match_primary


,movie,movie_country,cites,cites_country,title,year,primaryTitle_citation,year_citation,title_citation_normalized,movie_normalized,...,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,originalTitle_normalized,primaryTitle_normalized,match_primaryTitle,year_diff
0,A Real Life!!! (2006),Germany,La più bella serata della mia vita (1972),Italy,A Real Life!!!,2006.0,La più bella serata della mia vita,1972.0,lapiubellaseratadellamiavita,areallife,...,movie,A Real Life,Au voleur,0,2009.0,"Crime,Drama,Romance",auvoleur,areallife,both,-3.0
1,A Real Life!!! (2006),Germany,La più bella serata della mia vita (1972),France,A Real Life!!!,2006.0,La più bella serata della mia vita,1972.0,lapiubellaseratadellamiavita,areallife,...,movie,A Real Life,Au voleur,0,2009.0,"Crime,Drama,Romance",auvoleur,areallife,both,-3.0
2,Angels Wash Their Faces (1939),USA,Code of the Streets (1939),USA,Angels Wash Their Faces,1939.0,Code of the Streets,1939.0,codeofthestreets,angelswashtheirfaces,...,movie,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,both,0.0
3,Angels Wash Their Faces (1939),USA,Hell's Kitchen (1939),USA,Angels Wash Their Faces,1939.0,Hell's Kitchen,1939.0,hellskitchen,angelswashtheirfaces,...,movie,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,both,0.0
4,Angels Wash Their Faces (1939),USA,They Made Me a Criminal (1939),USA,Angels Wash Their Faces,1939.0,They Made Me a Criminal,1939.0,theymademeacriminal,angelswashtheirfaces,...,movie,Angels Wash Their Faces,The Angels Wash Their Faces,0,1939.0,"Drama,Romance",theangelswashtheirfaces,angelswashtheirfaces,both,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
309,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0
310,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0
311,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,xmenthelaststand,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0
312,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,xfirstclass,xmendarkphoenix,...,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",darkphoenix,xmendarkphoenix,both,-1.0


In [32]:
#get final data for movies
final_movies = pd.concat([matches_movies,movies_match_primary],ignore_index=True)
final_movies['match_primaryTitle'].replace({'both': 'True'},inplace=True)
final_movies['match_primaryTitle'] = final_movies['match_primaryTitle'].cat.add_categories(['False']).fillna('False')
final_movies.columns = final_movies.columns.str.replace("_citation", "_cit")
final_movies.drop(columns=['title_cit_normalized','movie_normalized','originalTitle_normalized','primaryTitle_normalized','_merge'], inplace=True)
final_movies

,movie,movie_country,cites,cites_country,title,year,primaryTitle_cit,year_cit,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,genres,year_diff,match_primaryTitle
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,2 Jennifer,2016.0,tt5611424,movie,From Jennifer,From Jennifer,0,2017.0,"Comedy,Horror,Thriller",0.0,False
1,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,To Jennifer,2013.0,tt5611424,movie,From Jennifer,From Jennifer,0,2017.0,"Comedy,Horror,Thriller",0.0,False
2,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,"Love, Simple",2012.0,tt1900964,movie,Rip,Rip,0,2010.0,Musical,3.0,False
3,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,"Love, Simple",2012.0,tt2290968,movie,R.I.P,R.I.P,0,2011.0,Action,2.0,False
4,R.I.P. (2013),USA,Doctor Faustus (1967),UK,R.I.P.,2013.0,Doctor Faustus,1967.0,tt1900964,movie,Rip,Rip,0,2010.0,Musical,3.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227889,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
227890,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
227891,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,X-Men: The Last Stand,2006.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True
227892,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,X: First Class,2011.0,tt6565702,movie,X-Men: Dark Phoenix,Dark Phoenix,0,2019.0,"Action,Adventure,Sci-Fi",-1.0,True


In [33]:
final_movies = final_movies.drop_duplicates()
final_citations = final_citations.drop_duplicates()

final_movies = final_movies.dropna(axis=1, how='all')

final_citations = final_citations.dropna(axis=1, how='all')




In [34]:
#remove unneeded columns
citations_data = final_citations[['movie', 'movie_country','cit','country_cit','primaryTitle_cit','year_cit','tconst_cit','titleType_cit','genres_cit']]

citations_data = citations_data.loc[:, ~citations_data.columns.duplicated()]
citations_data


movies_data  = final_movies[['movie', 'movie_country','cites','cites_country','title','year','tconst','titleType','genres']]

movies_data



,movie,movie_country,cites,cites_country,title,year,tconst,titleType,genres
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller"
1,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller"
2,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt1900964,movie,Musical
3,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt2290968,movie,Action
4,R.I.P. (2013),USA,Doctor Faustus (1967),UK,R.I.P.,2013.0,tt1900964,movie,Musical
...,...,...,...,...,...,...,...,...,...
227889,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"
227890,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"
227891,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"
227892,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"


As a last step, we combine the dataframes created above to get the final data for our graphs:
    all relevant information for movies: titles, country, year, genre, type, imdb identifier.

In [35]:

graph_df = pd.merge(movies_data, citations_data, left_on=['movie','movie_country','cites','cites_country'],
                    right_on = ['movie','movie_country','cit','country_cit'])

graph_df

,movie,movie_country,cites,cites_country,title,year,tconst,titleType,genres,cit,country_cit,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,genres_cit
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),USA,2 Jennifer,2016.0,tt4627224,movie,Horror
1,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",To Jennifer (2013),USA,To Jennifer,2013.0,tt2630300,movie,"Horror,Thriller"
2,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt1900964,movie,Musical,"Love, Simple (2012)",USA,"Love, Simple",2012.0,tt1153698,movie,"Comedy,Romance"
3,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt2290968,movie,Action,"Love, Simple (2012)",USA,"Love, Simple",2012.0,tt1153698,movie,"Comedy,Romance"
4,R.I.P. (2013),USA,Doctor Faustus (1967),UK,R.I.P.,2013.0,tt1900964,movie,Musical,Doctor Faustus (1967),UK,Doctor Faustus,1967.0,tt0062898,movie,"Drama,Horror,Mystery"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219095,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: The Last Stand (2006),Canada,X-Men: The Last Stand,2006.0,tt0376994,movie,"Action,Adventure,Sci-Fi"
219096,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: The Last Stand (2006),USA,X-Men: The Last Stand,2006.0,tt0376994,movie,"Action,Adventure,Sci-Fi"
219097,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: The Last Stand (2006),UK,X-Men: The Last Stand,2006.0,tt0376994,movie,"Action,Adventure,Sci-Fi"
219098,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),USA,X: First Class,2011.0,tt1270798,movie,"Action,Sci-Fi"


In [36]:
graph_df['movie'].unique()


<StringArray>
[                                       '#FromJennifer (2017)',
                                                 '#Rip (2013)',
                                               'R.I.P. (2013)',
                                         '#TubeClash02 (2016)',
 '#chicagoGirl: The Social Network Takes on a Dictator (2013)',
                                            '$10 Raise (1935)',
                                             '$5 a Day (2008)',
                                           '$ellebrity (2012)',
                                              '$windle (2002)',
                               "'94 du bi dao zhi qing (1994)",
 ...
                                   'Tea with Mussolini (1999)',
                    "The Crime Doctor's Strangest Case (1943)",
                                       'The Incomplete (2013)',
                                       'The Land of Oz (2015)',
                            'The Prince and the Dybbuk (2017)',
                     

In [40]:
graph_df.to_csv(path+'/datasets/graph_1_movies_cit.csv')

To retrieve film agents, we will use the tconst obtained for each film and its citations to link the data with the people involved in each film as directors, writers or producers, from the imdb dataset.

In [41]:
#get directors, writers, producers of films

movies_data


,movie,movie_country,cites,cites_country,title,year,tconst,titleType,genres
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller"
1,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller"
2,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt1900964,movie,Musical
3,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt2290968,movie,Action
4,R.I.P. (2013),USA,Doctor Faustus (1967),UK,R.I.P.,2013.0,tt1900964,movie,Musical
...,...,...,...,...,...,...,...,...,...
227889,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),Canada,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"
227890,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"
227891,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"
227892,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi"


In [42]:
# isolate the positions we are interested in, filtering out actors and other members.
crew_table = title_table[title_table['category'].isin(['director', 'producer', 'writer']) ]

#perform left merge to get the needed data for our films.
crew_data = pd.merge(graph_df, crew_table, left_on='tconst', right_on='tconst', how='left')
crew_data

,movie,movie_country,cites,cites_country,title,year,tconst,titleType,genres,cit,...,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,genres_cit,ordering,nconst,category,job,characters
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,2 Jennifer,2016.0,tt4627224,movie,Horror,5.0,nm3106201,director,\N,\N
1,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,2 Jennifer,2016.0,tt4627224,movie,Horror,6.0,nm4097598,writer,based on characters by,\N
2,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,2 Jennifer,2016.0,tt4627224,movie,Horror,7.0,nm3359084,producer,producer,\N
3,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",To Jennifer (2013),...,To Jennifer,2013.0,tt2630300,movie,"Horror,Thriller",5.0,nm3106201,director,\N,\N
4,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",To Jennifer (2013),...,To Jennifer,2013.0,tt2630300,movie,"Horror,Thriller",6.0,nm4097598,writer,based on characters by,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
804443,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,X: First Class,2011.0,tt1270798,movie,"Action,Sci-Fi",8.0,nm0795682,producer,producer,\N
804444,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,X: First Class,2011.0,tt1270798,movie,"Action,Sci-Fi",5.0,nm1334526,director,\N,\N
804445,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,X: First Class,2011.0,tt1270798,movie,"Action,Sci-Fi",6.0,nm0356745,producer,producer,\N
804446,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,X: First Class,2011.0,tt1270798,movie,"Action,Sci-Fi",7.0,nm0404446,producer,producer,\N


In [43]:
len(crew_data[crew_data['tconst'].isna()])

0

In [44]:
#missing attributes
len(crew_data[crew_data['nconst'].isna()]) 

5092

The missing data is due to the fact that some movies do not have all listed positions for the films

In [45]:
crew_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 804448 entries, 0 to 804447
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   movie             804448 non-null  string 
 1   movie_country     803721 non-null  string 
 2   cites             804448 non-null  string 
 3   cites_country     804382 non-null  string 
 4   title             804448 non-null  string 
 5   year              804448 non-null  float64
 6   tconst            804448 non-null  object 
 7   titleType         804448 non-null  string 
 8   genres            798475 non-null  string 
 9   cit               804448 non-null  string 
 10  country_cit       804382 non-null  string 
 11  primaryTitle_cit  804448 non-null  string 
 12  year_cit          804448 non-null  float64
 13  tconst_cit        804448 non-null  string 
 14  titleType_cit     804448 non-null  string 
 15  genres_cit        794099 non-null  string 
 16  ordering          79

In [46]:
title_table[title_table['tconst'] == 'tt6231178']
#while the data has info for actors, it does not have any for our interested positions.

,tconst,ordering,nconst,category,job,characters
44884253,tt6231178,1,nm2100606,actress,\N,"[""Melody Casting""]"
44884254,tt6231178,2,nm2153041,actor,\N,"[""Devin""]"


Perform a second merge to get the info for our citations as well.

In [47]:
crew_df = pd.merge(crew_data,crew_table,left_on='tconst_cit', right_on='tconst', how='left')
crew_df

,movie,movie_country,cites,cites_country,title,year,tconst_x,titleType,genres,cit,...,nconst_x,category_x,job_x,characters_x,tconst_y,ordering_y,nconst_y,category_y,job_y,characters_y
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,5.0,nm4097598,writer,characters,\N
1,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,6.0,nm1735659,producer,producer,\N
2,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,7.0,nm4259291,producer,producer,\N
3,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,8.0,nm3106201,producer,producer,\N
4,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm4097598,writer,based on characters by,\N,tt4627224,5.0,nm4097598,writer,characters,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291414,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,5.0,nm0891216,director,\N,\N
3291415,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,6.0,nm1005420,writer,screenplay by,\N
3291416,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,7.0,nm0826714,writer,screenplay by,\N
3291417,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,8.0,nm0963359,writer,screenplay by,\N


In [48]:
#cleanup to view which info is for movies (_'movie') and citations('_cit').
crew_df.columns = crew_df.columns.str.replace("_y", "_cit")
crew_df.columns = crew_df.columns.str.replace("_x", "_movie")

crew_df

,movie,movie_country,cites,cites_country,title,year,tconst_movie,titleType,genres,cit,...,nconst_movie,category_movie,job_movie,characters_movie,tconst_cit,ordering_cit,nconst_cit,category_cit,job_cit,characters_cit
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,5.0,nm4097598,writer,characters,\N
1,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,6.0,nm1735659,producer,producer,\N
2,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,7.0,nm4259291,producer,producer,\N
3,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm3106201,director,\N,\N,tt4627224,8.0,nm3106201,producer,producer,\N
4,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,nm4097598,writer,based on characters by,\N,tt4627224,5.0,nm4097598,writer,characters,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291414,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,5.0,nm0891216,director,\N,\N
3291415,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,6.0,nm1005420,writer,screenplay by,\N
3291416,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,7.0,nm0826714,writer,screenplay by,\N
3291417,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,nm0795682,producer,producer,\N,tt1270798,8.0,nm0963359,writer,screenplay by,\N


In [49]:
crew_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3291419 entries, 0 to 3291418
Data columns (total 27 columns):
 #   Column            Dtype  
---  ------            -----  
 0   movie             string 
 1   movie_country     string 
 2   cites             string 
 3   cites_country     string 
 4   title             string 
 5   year              float64
 6   tconst_movie      object 
 7   titleType         string 
 8   genres            string 
 9   cit               string 
 10  country_cit       string 
 11  primaryTitle_cit  string 
 12  year_cit          float64
 13  tconst_cit        object 
 14  titleType_cit     string 
 15  genres_cit        string 
 16  ordering_movie    float64
 17  nconst_movie      object 
 18  category_movie    object 
 19  job_movie         object 
 20  characters_movie  object 
 21  tconst_cit        object 
 22  ordering_cit      float64
 23  nconst_cit        object 
 24  category_cit      object 
 25  job_cit           object 
 26  characters_cit

In [50]:
#remove unneccessary columns
crew_df.drop(columns=['ordering_movie','ordering_cit','characters_movie', 'characters_cit'], inplace=True)
crew_df

,movie,movie_country,cites,cites_country,title,year,tconst_movie,titleType,genres,cit,...,tconst_cit,titleType_cit,genres_cit,nconst_movie,category_movie,job_movie,tconst_cit,nconst_cit,category_cit,job_cit
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm4097598,writer,characters
1,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm1735659,producer,producer
2,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm4259291,producer,producer
3,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm3106201,producer,producer
4,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm4097598,writer,based on characters by,tt4627224,nm4097598,writer,characters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291414,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm0891216,director,\N
3291415,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm1005420,writer,screenplay by
3291416,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm0826714,writer,screenplay by
3291417,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),UK,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm0963359,writer,screenplay by


In [51]:
crew_df.to_csv(path+'/datasets/crew_data.csv')

In [52]:
crew_df[crew_df.duplicated()]


,movie,movie_country,cites,cites_country,title,year,tconst_movie,titleType,genres,cit,...,tconst_cit,titleType_cit,genres_cit,nconst_movie,category_movie,job_movie,tconst_cit,nconst_cit,category_cit,job_cit


Finally we can filter our results to isolate our project's target countries

In [53]:
target_countries = ['USA', 'China', 'Hong Kong', 'Japan', 'South Korea', 'Taiwan', 'Mongolia', 'North Korea']

crew_df_target = crew_df[(crew_df['movie_country'].isin(target_countries)) & (crew_df['cites_country'].isin(target_countries))]
crew_df_target



,movie,movie_country,cites,cites_country,title,year,tconst_movie,titleType,genres,cit,...,tconst_cit,titleType_cit,genres_cit,nconst_movie,category_movie,job_movie,tconst_cit,nconst_cit,category_cit,job_cit
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm4097598,writer,characters
1,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm1735659,producer,producer
2,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm4259291,producer,producer
3,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm3106201,director,\N,tt4627224,nm3106201,producer,producer
4,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),...,tt4627224,movie,Horror,nm4097598,writer,based on characters by,tt4627224,nm4097598,writer,characters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3291390,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm0891216,director,\N
3291391,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm1005420,writer,screenplay by
3291392,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm0826714,writer,screenplay by
3291393,X-Men: Dark Phoenix (2018),USA,X: First Class (2011),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X: First Class (2011),...,tt1270798,movie,"Action,Sci-Fi",nm0795682,producer,producer,tt1270798,nm0963359,writer,screenplay by


In [54]:
graph_df_target = graph_df[(graph_df['movie_country'].isin(target_countries)) & (graph_df['cites_country'].isin(target_countries))]
graph_df_target

,movie,movie_country,cites,cites_country,title,year,tconst,titleType,genres,cit,country_cit,primaryTitle_cit,year_cit,tconst_cit,titleType_cit,genres_cit
0,#FromJennifer (2017),USA,2 Jennifer (2016),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",2 Jennifer (2016),USA,2 Jennifer,2016.0,tt4627224,movie,Horror
1,#FromJennifer (2017),USA,To Jennifer (2013),USA,#FromJennifer,2017.0,tt5611424,movie,"Comedy,Horror,Thriller",To Jennifer (2013),USA,To Jennifer,2013.0,tt2630300,movie,"Horror,Thriller"
2,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt1900964,movie,Musical,"Love, Simple (2012)",USA,"Love, Simple",2012.0,tt1153698,movie,"Comedy,Romance"
3,#Rip (2013),USA,"Love, Simple (2012)",USA,#Rip,2013.0,tt2290968,movie,Action,"Love, Simple (2012)",USA,"Love, Simple",2012.0,tt1153698,movie,"Comedy,Romance"
7,#chicagoGirl: The Social Network Takes on a Di...,USA,The Godfather (1972),USA,#chicagoGirl: The Social Network Takes on a Di...,2013.0,tt2607968,movie,"Action,Biography,Documentary",The Godfather (1972),USA,The Godfather,1972.0,tt0068646,movie,"Crime,Drama"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219089,X-Men: Dark Phoenix (2018),USA,X-Men Origins: Wolverine (2009),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men Origins: Wolverine (2009),USA,X-Men Origins: Wolverine,2009.0,tt0458525,movie,"Action,Sci-Fi"
219091,X-Men: Dark Phoenix (2018),USA,X-Men: Apocalypse (2016),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: Apocalypse (2016),USA,X-Men: Apocalypse,2016.0,tt3385516,movie,"Action,Adventure,Sci-Fi"
219092,X-Men: Dark Phoenix (2018),USA,X-Men: Days of Future Past (2014),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: Days of Future Past (2014),USA,X-Men: Days of Future Past,2014.0,tt1877832,movie,"Action,Adventure,Sci-Fi"
219096,X-Men: Dark Phoenix (2018),USA,X-Men: The Last Stand (2006),USA,X-Men: Dark Phoenix,2018.0,tt6565702,movie,"Action,Adventure,Sci-Fi",X-Men: The Last Stand (2006),USA,X-Men: The Last Stand,2006.0,tt0376994,movie,"Action,Adventure,Sci-Fi"


In [55]:
graph_df_target['movie_country'].unique()
crew_df_target['cites_country'].unique()

<StringArray>
['USA', 'Hong Kong', 'Japan', 'South Korea', 'Taiwan', 'China', 'Mongolia']
Length: 7, dtype: string

In [56]:
graph_df_target.to_csv(path+"/datasets/filtered_graph1_data.csv")
crew_df_target.to_csv(path+"/datasets/filtered_graph2_data.csv")